## Zadanie optymalizacyjne polegające na dobraniu odpowiedniej wartości natężenia prądu źródła prądowego dla podanych ograniczeń maksymalnej dopuszczalnej wartości prądu na rezystorach

## Dołączanie potrzebnych bibliotek

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pulp
import pandas as pd

## Tworzenie suwaków dla  wartości rezystancji elementów i maksymalnej dopuszczalnej wartości   natężenia prądów dla tych elementów

In [2]:
style = {'description_width': 'initial'}
rezystancja_R1_slider = FloatSlider(min=0,max = 15, value= 8, description="Rezystancja R1", style= style )
rezystancja_R2_slider = FloatSlider(min=0,max = 15, value= 6, description="Rezystancja R2", style= style )
rezystancja_R4_slider = FloatSlider(min=0,max = 15, value= 10, description="Rezystancja R4", style= style )
rezystancja_R5_slider = FloatSlider(min=0,max = 15, value= 8, description="Rezystancja R5", style= style )
prad_R1_slider = FloatSlider(min=0,max = 5, value= 2, description="Prad R1", style= style )
prad_R2_slider = FloatSlider(min=0,max = 5, value= 3, description="Prad R2", style= style )
prad_R3_slider = FloatSlider(min=0,max = 5, value= 4, description="Prad R3", style= style )
prad_R4_slider = FloatSlider(min=0,max = 5, value= 2, description="Prad R4", style= style )
prad_R5_slider = FloatSlider(min=0,max = 5, value= 2, description="Prad R5", style= style )

## Definicja funkcji optymalizującej 

In [3]:
def rezystory_wartosci(
    rezystancja_R1 = 8,
    rezystancja_R2 = 6,
    rezystancja_R4 = 10,
    rezystancja_R5 = 8,
    prad_R1 = 2,
    prad_R2 = 3,
    prad_R3 = 4,
    prad_R4 = 2,
    prad_R5 = 2):
    problem_rezystory = pulp.LpProblem("Problem rezystory", pulp.LpMaximize)

    i1 = pulp.LpVariable('i1', lowBound=0, cat='Continuous')
    i2 = pulp.LpVariable('i2', lowBound=0, cat='Continuous')
    i3 = pulp.LpVariable('i3', lowBound=0, cat='Continuous')
    i4 = pulp.LpVariable('i4', lowBound=0, cat='Continuous')
    i5 = pulp.LpVariable('i5', lowBound=0, cat='Continuous')

    problem_rezystory += i3

    problem_rezystory += i1 <= prad_R1, "Dopuszczalny prąd R1"
    problem_rezystory += i2 <= prad_R2, "Dopuszczalny prąd R2"
    problem_rezystory += i3 <= prad_R3, "Dopuszczalny prąd R3"
    problem_rezystory += i4 <= prad_R4, "Dopuszczalny prąd R4"
    problem_rezystory += i5 <= prad_R5, "Dopuszczalny prąd R5"
    problem_rezystory += i1 + i2 == i3
    problem_rezystory += i3 == i4 + i5
    problem_rezystory += i1 * rezystancja_R1 == i2 * rezystancja_R2
    problem_rezystory += i4 * rezystancja_R4 == i5 * rezystancja_R5
    
    problem_rezystory.solve()
    print("Status:", pulp.LpStatus[problem_rezystory.status])
    for variable in problem_rezystory.variables():
        print("{} = {}".format(variable.name, variable.varValue))
    print("Prąd wyniesie {} A  ".format(pulp.value(problem_rezystory.objective)))
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in problem_rezystory.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)
    print(pd.DataFrame(shadows))


## Wyświetlanie suwaków i wywoływanie funkcji rezystory_wartosci 

In [4]:
interact(rezystory_wartosci, rezystancja_R1 = rezystancja_R1_slider,
         rezystancja_R2 = rezystancja_R2_slider,
         rezystancja_R4 = rezystancja_R4_slider,
         rezystancja_R5 = rezystancja_R5_slider,
         prad_R1 = prad_R1_slider,
         prad_R2 = prad_R2_slider,
         prad_R3 = prad_R3_slider,
         prad_R4 = prad_R4_slider,
         prad_R5 = prad_R5_slider
 );

Status: Optimal
i1 = 1.5428571
i2 = 2.0571429
i3 = 3.6
i4 = 1.6
i5 = 2.0
Prąd wyniesie 3.6 A  
**********Shadow Prices**********
                   name  shadow price         slack
0  Dopuszczalny_prąd_R1          -0.0  4.571429e-01
1  Dopuszczalny_prąd_R2          -0.0  9.428571e-01
2  Dopuszczalny_prąd_R3          -0.0  4.000000e-01
3  Dopuszczalny_prąd_R4          -0.0  4.000000e-01
4  Dopuszczalny_prąd_R5           1.8 -0.000000e+00
5                   _C1           0.0 -4.440892e-16
6                   _C2           1.0 -0.000000e+00
7                   _C3          -0.0 -0.000000e+00
8                   _C4           0.1 -0.000000e+00
